In [ ]:
import numpy

In [ ]:
file=读取./raw下的所有csv文件
print 列名
如果该列列名有功能段，则作为target，然后另外一列非Title的作为input，按照gpt微调格式填充：
<s>Human: Write the '$文件名（去除后缀）' with the information below:{input} </s><s>Assistant: {target} </s>


In [39]:
import os
import pandas as pd

# Define the path to the directory containing the raw CSV files
raw_dir_path = './raw'
clean_dir_path = './clean'

# Ensure the clean directory exists
os.makedirs(clean_dir_path, exist_ok=True)

# Iterate over each file in the raw directory
for filename in os.listdir(raw_dir_path):
    # Check if the file is a CSV file
    if filename.endswith('.csv'):
        # Read the CSV file
        file_path = os.path.join(raw_dir_path, filename)
        df = pd.read_csv(file_path)

        # Print column names
        print(f"File: {filename}, Columns: {df.columns.tolist()}")

        # Identify the target and input columns
        target_column = None
        input_column = None
        for column in df.columns:
            if "过渡段" in column or "功能段" in column:
                target_column = column
            elif column.lower() != "title":
                input_column = column

        # If target and input columns are identified, proceed with formatting
        if target_column and input_column:
            # Create a new DataFrame for the clean data
            clean_df = pd.DataFrame({
                "text": f"<s>Human: Write the '{filename.replace('.csv', '')}' with the information below:" + df[input_column] + " </s><s>Assistant: " + df[target_column].astype(str) + " </s>"
            })

            # Write the clean DataFrame to a new CSV file in the clean directory
            clean_file_path = os.path.join(clean_dir_path, filename)
            clean_df.to_csv(clean_file_path, index=False)
        else:
            print(f"Could not identify target and input columns for file: {filename}")


File: research design.csv, Columns: ['Title', 'Research Desgin 研究设计功能段', 'research design']
File: transition_background.csv, Columns: ['Title', 'Transition I 背景类过渡段', 'Background information']
File: research findings.csv, Columns: ['Title', 'Research Finding 研究发现功能段', 'Findings']
File: transition program.csv, Columns: ['Unnamed: 0', 'Title', 'Transition II 项目类过渡段', 'program information']


In [ ]:
file=读取./clean下的所有csv文件
对每列所有数据，如果数据的字符串中有'Identification_strategy'，则用replace方法替换为''
同时，去除字符串中的6:这种数字+:的内容。
写入./clean_again下的csv文件，各自保存
然后将csv合并。写入final.csv

In [41]:
import pandas as pd
import os
import re

# 确保输出目录存在
output_dir = './clean_again'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 读取./clean目录下的所有csv文件
csv_files = [f for f in os.listdir('./clean') if f.endswith('.csv')]

# 初始化一个空的DataFrame来存储最终合并的数据
final_df = pd.DataFrame()

for file in csv_files:
    # 读取每个CSV文件
    df = pd.read_csv(os.path.join('./clean', file))
    
    # 对DataFrame的每个元素应用替换规则
    for col in df.columns:
        df[col] = df[col].astype(str).apply(lambda x: x.replace('Identification_strategy', ''))
        df[col] = df[col].apply(lambda x: re.sub(r'\d+:', '', x))
        df[col] = df[col].apply(lambda x: re.sub(r'\d+：', '', x))
    
    # 保存处理后的文件
    df.to_csv(os.path.join(output_dir, file), index=False)
    
    # 累加到最终的DataFrame
    final_df = pd.concat([final_df, df], ignore_index=True)

# 将合并后的DataFrame保存为final.csv
final_df.to_csv('final.csv', index=False)


In [1]:
import pandas as pd

# 读取final.csv文件
df = pd.read_csv('final.csv')

# 删除包含NaN值的行
df = df.dropna()

# 将结果写回到同一文件
df.to_csv('final_new.csv', index=False)


In [ ]:
Identification_strategy

In [37]:
text=df.iloc[10]['program information']

In [38]:
text

'"Program name": Defense Meteorological Satellite Program (DMSP)\n"Enforcer": Not specified\n"Beginning date": 1992\n"Main purpose": Originally designed to detect clouds for short-term Air Force weather forecasts\n"Development stage": No longer in operation, data stopped in 2013\n\n"Program name": Visible Infrared Imaging Radiometer Suite (VIIRS)\n"Enforcer": Not specified\n"Beginning date": Not specified\n"Main purpose": Consistently measure the radiance of light coming from earth, with high spatial accuracy and temporally comparable data\n"Development stage": Currently in use, with monthly data available\n\n"Program name": Night lights data\n"Enforcer": Not specified\n"Beginning date": Not specified\n"Main purpose": Proxy for local economic activity, particularly in poor countries\n"Development stage": Widely used in economics, with over 150 studies utilizing night lights data\n\n'

In [26]:
# Split the original text by ":", then for each segment, isolate the phrase immediately preceding the ":".
# This time, we need to ensure that phrases like "Enforcer" and "Main purpose" are correctly extracted as separate phrases.

# Define a function to correct and isolate phrases accurately
def correct_and_isolate_phrases(text):
    phrases = []  # List to hold the corrected phrases
    # Split the text by ":", to get segments containing the phrases of interest
    segments = text.split(":")
    
    for segment in segments[:-1]:  # Exclude the last segment as it doesn't precede a colon
        # Attempt to isolate the phrase by finding the last occurrence of certain keywords or capital letters
        # that typically start a phrase of interest
        potential_starts = [segment.rfind("Program"), segment.rfind("Enforcer"), segment.rfind("Beginning"), segment.rfind("Main"), segment.rfind("Development"), segment.rfind("Other")]
        # Filter out -1's (indicating no match) and find the maximum index to ensure we start from the latest possible point
        start_index = max([index for index in potential_starts if index != -1])
        if start_index != -1:
            # Extract the phrase from the start index to the end of the segment
            phrases.append(segment[start_index:].strip())
        else:
            # If no specific start is found, append the whole segment as a fallback
            phrases.append(segment.strip())
    
    return phrases

# Use the function to correct and isolate phrases from the text
corrected_phrases = correct_and_isolate_phrases(text)

# Join the corrected phrases with a newline character
formatted_text_final = "\n".join(corrected_phrases)

print(formatted_text_final)


Program name"
Enforcer"
Beginning date"
Main purpose"
Development stage"
Program name"
Enforcer"
Beginning date"
Main purpose"
Development stage"
Program name"
Enforcer"
Beginning date"
Main purpose"
Development stage"


In [14]:
clean_df.iloc[1]['text']

"<s>Human: Write the 'transition program' with the information below:Program name: United States Acid Rain Program (ARP)Enforcer: Environmental Protection Agency (EPA)Beginning date: 1995Main purpose: To control sulfur dioxide (SO2) emissions and reduce particulate matter smaller than 2.5 micrometers (PM2.5) pollution.Development stage: Implemented and enforced.Other information: The ARP created an immediate and persistent drop in SO2 emissions, allowing for an event study analysis to explore dynamic effects. The program regulated only certain SO2-intensive coal plants, comparing changes in mortality over time in counties most impacted by pollution from regulated plants to changes in mortality in counties with little to no related exposure. The program aimed to mitigate bias from national trends changing over time independent of pollution from coal-fired plants. The ARP generated limited economic spillovers and had negligible effects on income, employment, or migration. The program's e

In [1]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('./raw/transition background.csv')

# 剔除 'Research Finding 研究发现功能段' 列
df.drop(columns=['Research Finding 研究发现功能段'], inplace=True)

# 打印剩余的列
print(df.head())

# 如果需要保存剔除列后的结果到新的CSV文件，可以使用以下代码
df.to_csv('./raw/new_transition_background.csv', index=False)


                                               Title  \
0  Exposure to collective gender-based violence c...   
1  Live tutoring calls did not improve learning d...   
2  Manufacturing employment and women’s agency: E...   
3  Social and financial incentives for overcoming...   
4  The political impacts of land expropriation in...   

                                 Transition I 背景类过渡段  \
0  3：\nThis paper studies the causal long-run imp...   
1  2:\nTo keep students engaged and learning duri...   
2  2：\nThe paper contributes to the sparse litera...   
3  2：\nSince these pathogens are communicable, a ...   
4  2:\nState expropriation is generally justified...   

                              Background information  
0  "1": "Globally, one in three women experience ...  
1  "1": "School closures due to the COVID-19 pand...  
2  "1": "The paper examines the impact of women's...  
3  "1": "Open defecation is practiced by one bill...  
4  "1": "State expropriation is common in human h..